<a href="https://colab.research.google.com/github/Msuban1/Msuban1.github.io/blob/main/WEEK_1_Project_(Cleaning_dataset).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# This installs the tools we need to work with Google Sheets
!pip install --upgrade gspread gspread-dataframe google-auth

In [14]:
import gspread
from google.colab import auth
from google.auth import default
from gspread_dataframe import get_as_dataframe, set_with_dataframe

# This asks for permission to access your Google account
auth.authenticate_user()

# This sets up the connection to Google Sheets
creds, _ = default()
gc = gspread.authorize(creds)


In [15]:
# Paste your Google Sheet link below
sheet_url = 'https://docs.google.com/spreadsheets/d/1kojKp5JqJS7O9LgK1t3E5ZtjuYGm95Qx2R7DetCVfXs/edit?gid=866428366#gid=866428366'

# Open the spreadsheet
spreadsheet = gc.open_by_url(sheet_url)

# Get the first worksheet (tab) in the spreadsheet
worksheet = spreadsheet.get_worksheet(0)

# Convert the worksheet to a pandas DataFrame so we can clean it
sales_df = get_as_dataframe(worksheet, dtype=str)

# Remove completely empty rows (just in case)
sales_df.dropna(how='all', inplace=True)

# Show the first few rows of data
sales_df.head()

,Transaction_ID,Date,Customer_ID,Product,Category,Quantity,Price,Total_Amount,Payment_Method,Region
0,1001,2024-01-05,C001,Laptop,Electronics,1,800,NaN,Credit Card,North
1,1002,2024-01-10,C002,Smartphone,Electronics,2,600,1200,Cash,South
2,1003,2024-01-12,C003,Headphones,Electronics,1,100,100,PayPal,West
3,1004,2024-02-05,C004,Tablet,Electronics,1,500,500,Debit Card,East
4,1005,2024-02-08,C005,Book,Books,3,20,60,Credit Card,North


# Task
Clean this dataset.

Here is all the data you need:
"working copy of sales_data.xlsx"

## Data cleaning

### Subtask:
Remove all duplicates from dataset.


**Reasoning**:
Inspect the data, cleaning the data collected by removing any duplicates to ensure a correct outcome and insight to make any business decision.



In [24]:
import pandas as pd

# Check how many duplicate rows there are
duplicate_count = sales_df.duplicated().sum()
print("Number of duplicate rows found:", duplicate_count)

# Remove duplicates directly in the dataset
sales_df.drop_duplicates(inplace=True)

# Show the new size of the dataset
print("Shape after removing duplicates:", sales_df.shape)

Number of duplicate rows found: 0
Shape after removing duplicates: (20, 10)


**Conclusion**: There was no duplicates found in the dataset.
The dataset is 20 rows, 10 columns.

***Saved cleaned duplicates dataset to Google Sheet***

In [29]:
from gspread_dataframe import set_with_dataframe

# Overwrite the original sheet with the cleaned data
set_with_dataframe(worksheet, sales_df)

print("Cleaned data (duplicates removed) saved to Google Sheets.")


Cleaned data (duplicates removed) saved to Google Sheets.


### Subtask 2:

Inspect the data for missing values.

In [17]:
# Show count of missing values per column
print("Missing values in each column:")
print(sales_df.isnull().sum())


Missing values in each column:
Transaction_ID    0
Date              0
Customer_ID       0
Product           0
Category          0
Quantity          0
Price             0
Total_Amount      1
Payment_Method    0
Region            0
dtype: int64


**Result**: 1 missing value was found in the Total_Amount column.

In [30]:
# Overwrite the worksheet with the cleaned data

from gspread_dataframe import set_with_dataframe
set_with_dataframe(worksheet, sales_df)

print("Missing values handled and cleaned data saved back to Google Sheets.")


Missing values handled and cleaned data saved back to Google Sheets.


### Subtask 3:
Replace missing values with mean or median. I inspected and cleaned the dataset separating it into two catergories - Numerical and Catergorical.

In [18]:
# Convert Price column to float and fill missing values with the average
sales_df['Price'] = sales_df['Price'].astype(float)
average_price = sales_df['Price'].mean()
sales_df['Price'] = sales_df['Price'].fillna(average_price)

# Convert Total_Amount to float and fill missing values with the average
sales_df['Total_Amount'] = sales_df['Total_Amount'].astype(float)
average_total = sales_df['Total_Amount'].mean()
sales_df['Total_Amount'] = sales_df['Total_Amount'].fillna(average_total)

print("Filled missing values in Price and Total_Amount with their averages.")



Filled missing values in Price and Total_Amount with their averages.


In [19]:
# Fill missing values in 'Category' with the most common value
most_common_category = sales_df['Category'].mode()[0]
sales_df['Category'] = sales_df['Category'].fillna(most_common_category)

# Fill missing values in 'Region' with the most common value
most_common_region = sales_df['Region'].mode()[0]
sales_df['Region'] = sales_df['Region'].fillna(most_common_region)

# Fill missing values in 'Payment_Method' with the most common value
most_common_payment = sales_df['Payment_Method'].mode()[0]
sales_df['Payment_Method'] = sales_df['Payment_Method'].fillna(most_common_payment)

print("Filled missing values in Category, Region, and Payment_Method with the most common value.")



Filled missing values in Category, Region, and Payment_Method with the most common value.


In [31]:
# Overwrite the worksheet with the cleaned data
set_with_dataframe(worksheet, sales_df)

print("Cleaned data has been saved back to your Google Sheet.")


Cleaned data has been saved back to your Google Sheet.


### Subtask 4:
Standardise Formats - This ensures data formats are consistent (e.g, dates in YYYY-MM-DD format, numbers are actually stored as numbers (not text) and numerical columns are without symbols).


In [25]:
# Convert the 'Date' column to datetime format
sales_df['Date'] = pd.to_datetime(sales_df['Date'], errors='coerce')

# Format the date column as 'YYYY-MM-DD'
sales_df['Date'] = sales_df['Date'].dt.strftime('%Y-%m-%d')

# Fill any failed conversions with 'n/a'
sales_df['Date'] = sales_df['Date'].fillna('n/a')

print("✅ Dates are now in YYYY-MM-DD format.")

✅ Dates are now in YYYY-MM-DD format.


In [26]:
# Convert numeric columns to the right format
sales_df['Price'] = pd.to_numeric(sales_df['Price'], errors='coerce')
sales_df['Quantity'] = pd.to_numeric(sales_df['Quantity'], errors='coerce')
sales_df['Total_Amount'] = pd.to_numeric(sales_df['Total_Amount'], errors='coerce')

# If any became NaN due to errors, fill with 0 or a default value
sales_df['Price'] = sales_df['Price'].fillna(0)
sales_df['Quantity'] = sales_df['Quantity'].fillna(0)
sales_df['Total_Amount'] = sales_df['Total_Amount'].fillna(0)

print("✅ Price, Quantity, and Total_Amount columns are now numeric.")

✅ Price, Quantity, and Total_Amount columns are now numeric.


### DATA CLEANED ✨

**Time to export it onto a google sheet file!**

In [32]:
# Save the fully cleaned and formatted dataset back to Google Sheets
from gspread_dataframe import set_with_dataframe
set_with_dataframe(worksheet, sales_df)

print("Final cleaned and formatted data saved to your Google Sheet.")


Final cleaned and formatted data saved to your Google Sheet.


## Data exploration

### Subtask:
Explore the loaded sales data to understand its characteristics.


**Reasoning**:
Explore the sales data by examining data types, missing values, descriptive statistics, unique values in categorical columns, and the overall shape of the DataFrame. Also, check for correlations between numerical variables.



In [ ]:
# Examine data types
print("Data Types:\n", sales_df.dtypes)

# Check for missing values
print("\nMissing Values:\n", sales_df.isnull().sum())

# Descriptive statistics for numerical columns
print("\nDescriptive Statistics:\n", sales_df.describe())

# Exclude 'Transaction_ID' and focus only on numeric columns
# Corrected: Use 'sales_df' instead of 'df'
df_numeric = sales_df.drop(columns=['Transaction_ID'], errors='ignore').select_dtypes(include='number')


# Unique values and frequencies for categorical columns
for col in ['Payment_Method', 'Region', 'Product', 'Category']:
    print(f"\nUnique values and frequencies for {col}:\n{sales_df[col].value_counts()}")

# Overall shape of the DataFrame
print(f"\nShape of the DataFrame: {sales_df.shape}")

# Correlation between numerical variables
print("\nCorrelation between numerical variables:\n",sales_df[['Quantity', 'Price', 'Total_Amount']].corr())

In [ ]:
!pip install --upgrade -q gspread gspread-dataframe


## Data visualization

### Subtask:
Visualize the distribution of key features in the sales data.


**Reasoning**:
Visualize the distribution of key features using histograms and bar charts.



In [ ]:
import matplotlib.pyplot as plt

# Histograms for numerical features
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.hist(sales_df['Quantity'], bins=5, edgecolor='black')
plt.xlabel('Quantity')
plt.ylabel('Frequency')
plt.title('Distribution of Quantity')

plt.subplot(1, 3, 2)
plt.hist(sales_df['Price'], bins=5, edgecolor='black')
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.title('Distribution of Price')

plt.subplot(1, 3, 3)
plt.hist(sales_df['Total_Amount'].dropna(), bins=5, edgecolor='black') # Handle missing values
plt.xlabel('Total Amount')
plt.ylabel('Frequency')
plt.title('Distribution of Total Amount')

plt.tight_layout()
plt.show()


# Bar charts for categorical features
categorical_features = ['Payment_Method', 'Region', 'Product', 'Category']
plt.figure(figsize=(15, 10))

for i, feature in enumerate(categorical_features):
    plt.subplot(2, 2, i + 1)
    sales_df[feature].value_counts().plot(kind='bar', color='skyblue')
    plt.xlabel(feature)
    plt.ylabel('Frequency')
    plt.title(f'Frequency of {feature}')
    plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.show()

## Data analysis

### Subtask:
Perform detailed data analysis on the sales data.


**Reasoning**:
Calculate descriptive statistics for numerical features, analyze categorical feature frequencies, explore correlations between numerical features, and investigate relationships between numerical and categorical features. Handle the missing 'Total_Amount' value before calculations.



In [ ]:
# Calculate descriptive statistics for numerical features
numerical_features = ['Quantity', 'Price', 'Total_Amount']
sales_df[numerical_features].describe()

# Handle missing value in 'Total_Amount' (using imputation for demonstration)
sales_df['Total_Amount'] = sales_df['Total_Amount'].fillna(sales_df['Total_Amount'].mean())

# Recalculate descriptive statistics after imputation
print(sales_df[numerical_features].describe())

# Analyze frequency distribution of categorical features
categorical_features = ['Payment_Method', 'Region', 'Product', 'Category']
for col in categorical_features:
    frequencies = sales_df[col].value_counts(normalize=True) * 100
    print(f"\nFrequency distribution for {col}:\n{frequencies}")

# Explore correlations between numerical features
correlation_matrix = sales_df[['Quantity', 'Price', 'Total_Amount']].corr()
print(f"\nCorrelation Matrix:\n{correlation_matrix}")

# Investigate relationships between numerical and categorical features
for col in categorical_features:
    print(f"\nAverage price and total amount by {col}:")
    print(sales_df.groupby(col)[['Price', 'Total_Amount']].mean())

## Summary:

### Q&A
There were no explicit questions in the task description. However, the analysis implicitly answers questions about the dataset's characteristics, distributions, and relationships between variables.


### Data Analysis Key Findings
* **Missing Data:** A missing value was found in the `Total_Amount` column, which was subsequently imputed using the mean value.
* **Data Types:** The 'Date' column was identified as an object data type and should be converted to datetime.
* **Correlation:** A strong positive correlation (0.83) exists between 'Price' and 'Total\_Amount', while a weak negative correlation (-0.42) exists between 'Quantity' and 'Price'.  The correlation between 'Quantity' and 'Total\_Amount' was weak (-0.17).
* **Categorical Distributions:** The analysis revealed the frequency distribution of each categorical feature (Payment\_Method, Region, Product, Category). For instance, the analysis of average price and total amount by product showed that laptops and smartphones have higher average prices and total amounts compared to books and t-shirts.


### Insights or Next Steps
* Convert the 'Date' column to datetime format for time-series analysis.
* Further investigate the missing `Total_Amount` value and consider alternative imputation methods or removal if appropriate.  The impact of the imputation on subsequent analysis should be considered.
* Explore the relationship between sales and different product categories or regions in more depth.
